In [1]:
from __future__ import print_function
import os, gym, time, glob, argparse, sys
import numpy as np
from scipy.signal import lfilter
from scipy.misc import imresize # preserves single-pixel info _unlike_ img = img[::2,::2]
import tensorflow as tf
os.environ['OMP_NUM_THREADS'] = '1'

In [3]:
# arguments
env_name = 'Pong-v0'
num_process = 20 # number of processes to train with
render = False # whether to render the environment or not
test = False # set lr=0, chooses most likely actions
rnn_steps = 20 # steps to train LSTM
lr = 1e-4
seed = 1 # random seed for reproducibility
gamma = 0.99 # reward discount factor
tau = 1.0 # generalized adwantage estimation discount
horizon = 0.99 # horizon for running averages
h_size = 256 # hidden size 
num_actions = gym.make(env_name).action_space.n 
# get the action space of this game
print(num_actions)
env = gym.make(env_name)
s = env.reset()
print(s.shape)

6
(210, 160, 3)


In [4]:
# calculate discounte rewards
discount = lambda x, gamma: lfilter([1],[1,-gamma],x[::-1])[::-1] # discounted rewards one liner

# preprocessing 210*160*3 to 80*80
prepro = lambda img: imresize(img[35:195].mean(2), (80,80)).astype(np.float32).reshape(1,80,80)/255.

def printlog(args, s, end='\n', mode='a'):
    print(s, end=end) ; f=open(args.save_dir+'log.txt',mode) ; f.write(s+'\n') ; f.close()

In [5]:
def PolicyNetwork(num_pixels, h_size): # an policy gardient network
    # input batch_size * 6400
    pixels = tf.placeholder(dtype=tf.float32, shape=(None, pixels_num))    
    actions = tf.placeholder(dtype=tf.float32, shape=(None,1))
    rewards = tf.placeholder(dtype=tf.float32, shape=(None,1))

    with tf.variable_scope('policy'):
    hidden = tf.layers.dense(pixels, hidden_units, activation=tf.nn.relu,\
            kernel_initializer = tf.contrib.layers.xavier_initializer())
    logits = tf.layers.dense(hidden, 1, activation=None,\
            kernel_initializer = tf.contrib.layers.xavier_initializer())

    out = tf.sigmoid(logits, name="sigmoid")
    cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(
        labels=actions, logits=logits, name="cross_entropy")
    loss = tf.reduce_sum(tf.multiply(rewards, cross_entropy, name="rewards"))

    # lr=1e-4
    lr=1e-3
    decay_rate=0.99
    opt = tf.train.RMSPropOptimizer(lr, decay=decay_rate).minimize(loss)
    # opt = tf.train.AdamOptimizer(lr).minimize(loss)

    tf.summary.histogram("hidden_out", hidden)
    tf.summary.histogram("logits_out", logits)
    tf.summary.histogram("prob_out", out)
    merged = tf.summary.merge_all()

    # grads = tf.gradients(loss, [hidden_w, logit_w])
    # return pixels, actions, rewards, out, opt, merged, grads
    return pixels, actions, rewards, out, opt, merged

        
